# SQLite Local Benchmarking with BenchBox

This notebook demonstrates benchmarking SQLite, the world's most widely deployed database engine.

**What you'll learn:**
- Running TPC-H benchmarks with SQLite
- Optimizing SQLite for analytical workloads
- Using indexes and query optimization
- Comparing SQLite with DuckDB for analytics
- Understanding SQLite's strengths and limitations

**Why SQLite?**
- **Ubiquitous**: Pre-installed on most systems, used by billions of devices
- **Zero-config**: No server, no setup - just a single file
- **Portable**: Database is a single file you can copy/move
- **Reliable**: Extensively tested, ACID-compliant
- **Free**: Public domain, no licensing costs

**Important Note:**
SQLite is optimized for **transactional workloads** (OLTP), not analytics (OLAP). For analytical workloads, DuckDB is typically 10-100x faster. This notebook helps you understand when to use each.

**Prerequisites:**
- Python 3.8+ (SQLite included in Python standard library)
- Sufficient disk space for test data (~100MB-10GB depending on scale)

**Estimated time:** 5-30 minutes (scale factor 0.01-1.0)

## 1. Installation & Setup

### Install Required Packages

Install BenchBox and visualization libraries. SQLite is included with Python!

In [ ]:
!pip install -q benchbox pandas matplotlib seaborn psutil

### Import Libraries

Import BenchBox components and SQLite.

In [ ]:
import sqlite3
import warnings
from datetime import datetime
from pathlib import Path

warnings.filterwarnings("ignore")

# BenchBox imports
import matplotlib.pyplot as plt
import numpy as np

# Visualization imports
import seaborn as sns

from benchbox.core.config import BenchmarkConfig, DatabaseConfig
from benchbox.core.results.exporter import ResultExporter
from benchbox.core.runner import LifecyclePhases, run_benchmark_lifecycle

# Check SQLite version
print(f"✅ SQLite {sqlite3.sqlite_version} (included with Python {sqlite3.version})")

# System monitoring
try:
    import psutil

    print("✅ psutil imported for system monitoring")
except ImportError:
    print("⚠️  psutil not available - install for system monitoring: pip install psutil")
    psutil = None

# Configure plotting
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")
%matplotlib inline

print("\n📦 All libraries imported successfully")

### Configure SQLite

SQLite performance depends heavily on configuration. Here are key settings:

**PRAGMAs for Analytics:**
```sql
PRAGMA journal_mode = WAL;          -- Write-Ahead Logging for better concurrency
PRAGMA synchronous = NORMAL;        -- Balance safety and speed
PRAGMA cache_size = -64000;         -- 64MB cache (negative = KB)
PRAGMA temp_store = MEMORY;         -- Keep temp data in memory
PRAGMA mmap_size = 30000000000;     -- 30GB memory-mapped I/O
```

**Important Notes:**
- SQLite is single-threaded (one write at a time)
- Best for datasets <1GB; DuckDB recommended for larger
- Analytical queries can be slow without proper indexes

In [ ]:
# Configure benchmark settings
config = {
    "database_file": "./benchmark_runs/sqlite/benchbox.db",
    "cache_size_mb": 64,  # Cache size in MB
    "use_wal": True,  # Write-Ahead Logging
    # Scale factors to test (keep small for SQLite)
    "scale_factors": [0.01, 0.1],  # 10MB, 100MB (1GB can be slow)
    # Output directory
    "output_dir": "./benchmark_results",
}

# Create directories
Path(config["database_file"]).parent.mkdir(parents=True, exist_ok=True)
Path(config["output_dir"]).mkdir(parents=True, exist_ok=True)

# Get system information
if psutil:
    total_ram = psutil.virtual_memory().total / (1024**3)  # GB
    available_ram = psutil.virtual_memory().available / (1024**3)  # GB
    print("💻 System Information:")
    print(f"   Total RAM: {total_ram:.1f} GB")
    print(f"   Available RAM: {available_ram:.1f} GB")
    print(f"   SQLite Cache: {config['cache_size_mb']} MB")
else:
    print("💻 System Information:")
    print(f"   SQLite Cache: {config['cache_size_mb']} MB")

print("\n✅ Configuration complete")
print(f"   Database: {config['database_file']}")
print(f"   WAL mode: {config['use_wal']}")
print(f"   Output directory: {config['output_dir']}")

print("\n⚠️  SQLite Performance Note:")
print("   SQLite is optimized for transactional workloads, not analytics.")
print("   Expect slower performance than DuckDB on analytical queries.")
print("   For large analytical datasets, consider DuckDB instead.")

### Test SQLite Connection

Verify SQLite is working and apply performance settings.

In [ ]:
try:
    # Connect to SQLite
    conn = sqlite3.connect(config["database_file"])
    cursor = conn.cursor()

    # Apply performance settings
    if config["use_wal"]:
        cursor.execute("PRAGMA journal_mode = WAL;")
    cursor.execute("PRAGMA synchronous = NORMAL;")
    cursor.execute(f"PRAGMA cache_size = -{config['cache_size_mb'] * 1024};")
    cursor.execute("PRAGMA temp_store = MEMORY;")
    cursor.execute("PRAGMA mmap_size = 30000000000;")  # 30GB

    # Check version
    version = cursor.execute("SELECT sqlite_version();").fetchone()[0]
    print("✅ Connected to SQLite")
    print(f"   Version: {version}")

    # Check applied settings
    journal_mode = cursor.execute("PRAGMA journal_mode;").fetchone()[0]
    cache_size = cursor.execute("PRAGMA cache_size;").fetchone()[0]

    print("\n⚙️  Current Settings:")
    print(f"   Journal mode: {journal_mode}")
    print(f"   Cache size: {abs(cache_size) / 1024:.1f} MB")

    # Check compile options
    print("\n🔧 Compile Options:")
    options = cursor.execute("PRAGMA compile_options;").fetchall()
    key_options = ["THREADSAFE", "ENABLE_FTS", "ENABLE_JSON", "MAX_LENGTH"]
    for (option,) in options:
        for key in key_options:
            if key in option:
                print(f"   {option}")

    # Simple test query
    result = cursor.execute("SELECT 42 as answer, 'SQLite' as database;").fetchone()
    print(f"\n✅ Test query successful: {result}")

    conn.close()
    print("\n✅ Connection test passed!")

except Exception as e:
    print(f"❌ Connection failed: {e}")
    raise

## 2. Quick Start Example

### Run TPC-H Power Test

Execute a TPC-H power test at scale factor 0.01 (10MB). This runs all 22 TPC-H queries sequentially.

**What happens:**
1. Generate TPC-H data (customer, orders, lineitem, etc.)
2. Create tables in SQLite
3. Load data from generated files
4. Execute 22 queries and measure performance

**Expected time:** ~2-5 minutes at SF 0.01 (slower than DuckDB)

**Note**: SQLite is single-threaded and not optimized for analytics. Queries will be slower than DuckDB or cloud warehouses.

In [ ]:
# Configure database connection
db_cfg = DatabaseConfig(type="sqlite", name="sqlite-local")
platform_cfg = {
    "database": config["database_file"],
    "pragmas": {
        "journal_mode": "WAL" if config["use_wal"] else "DELETE",
        "synchronous": "NORMAL",
        "cache_size": -(config["cache_size_mb"] * 1024),
        "temp_store": "MEMORY",
    },
}

# Configure TPC-H benchmark
bench_cfg = BenchmarkConfig(
    name="tpch", display_name="TPC-H Power Test", scale_factor=0.01, test_execution_type="power"
)

# Track start time
start_time = datetime.now()

# Run complete lifecycle
print("🚀 Starting TPC-H power test on SQLite...\n")
print("⏱️  Note: SQLite is single-threaded and optimized for OLTP.")
print("   Analytical queries may take longer than DuckDB.\n")

results = run_benchmark_lifecycle(
    benchmark_config=bench_cfg,
    database_config=db_cfg,
    system_profile=None,
    platform_config=platform_cfg,
    phases=LifecyclePhases(generate=True, load=True, execute=True),
)

end_time = datetime.now()
total_time = (end_time - start_time).total_seconds()

print("\n✅ TPC-H power test completed!")
print(f"   Benchmark: {results.benchmark_name}")
print(f"   Total queries: {len(results.query_results)}")
print(f"   Geometric mean: {results.geometric_mean:.3f}s")
print(f"   Total execution time: {results.total_execution_time:.2f}s")
print(f"   Wall clock time: {total_time:.2f}s")

### Visualize Results

Create a bar chart showing execution time for each query.

In [ ]:
if results.query_results:
    query_names = [qr.query_name for qr in results.query_results]
    execution_times = [qr.execution_time for qr in results.query_results]

    fig, ax = plt.subplots(figsize=(14, 6))
    bars = ax.bar(query_names, execution_times, color="#003B57", alpha=0.8, edgecolor="black")

    # Highlight slowest queries
    max_time = max(execution_times)
    for i, (bar, time) in enumerate(zip(bars, execution_times)):
        if time > max_time * 0.7:  # Top 30% slowest
            bar.set_color("#0F80AA")  # SQLite blue accent
            # Annotate with time
            ax.text(i, time + max_time * 0.02, f"{time:.2f}s", ha="center", va="bottom", fontsize=8)

    ax.set_xlabel("Query", fontsize=12, fontweight="bold")
    ax.set_ylabel("Execution Time (seconds)", fontsize=12, fontweight="bold")
    ax.set_title("TPC-H Query Performance on SQLite (SF 0.01)", fontsize=14, fontweight="bold")
    ax.grid(axis="y", alpha=0.3)
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

    print("\n📊 Performance Summary:")
    print(f"   Fastest query: {query_names[execution_times.index(min(execution_times))]} ({min(execution_times):.3f}s)")
    print(f"   Slowest query: {query_names[execution_times.index(max(execution_times))]} ({max(execution_times):.3f}s)")
    print(f"   Median time: {sorted(execution_times)[len(execution_times) // 2]:.3f}s")

    # Calculate queries per second
    qps = len(execution_times) / results.total_execution_time
    print(f"   Throughput: {qps:.2f} queries/second")
else:
    print("⚠️  No query results to visualize")

### Monitor Resource Usage

Check system resource consumption during the benchmark.

In [ ]:
if psutil:
    # Get current resource usage
    cpu_percent = psutil.cpu_percent(interval=1)
    memory = psutil.virtual_memory()
    disk = psutil.disk_usage(".")

    print("💻 Resource Usage:\n")
    print(f"CPU Usage: {cpu_percent}%")
    print(f"Memory: {memory.used / (1024**3):.1f} GB / {memory.total / (1024**3):.1f} GB ({memory.percent}%)")
    print(f"Disk: {disk.used / (1024**3):.1f} GB / {disk.total / (1024**3):.1f} GB ({disk.percent}%)")

    print("\n💡 SQLite Resource Notes:")
    print("   - SQLite is single-threaded (won't use all CPU cores)")
    print("   - Low memory footprint compared to other databases")
    print("   - I/O bound for analytical queries")
else:
    print("⚠️  Resource monitoring not available (install psutil)")

# Check database file size
if Path(config["database_file"]).exists():
    db_size = Path(config["database_file"]).stat().st_size / (1024**2)

    # Check for WAL files
    wal_file = Path(str(config["database_file"]) + "-wal")
    shm_file = Path(str(config["database_file"]) + "-shm")

    print("\n💾 Database Files:")
    print(f"   Main database: {db_size:.1f} MB")
    if wal_file.exists():
        wal_size = wal_file.stat().st_size / (1024**2)
        print(f"   WAL file: {wal_size:.1f} MB")
    if shm_file.exists():
        shm_size = shm_file.stat().st_size / (1024**2)
        print(f"   SHM file: {shm_size:.1f} MB")

### Results Overview

Display detailed results including per-query breakdown.

In [ ]:
print("📊 Detailed Results:\n")
print(f"Benchmark: {results.benchmark_name}")
print(f"Platform: {results.platform}")
print(f"Scale Factor: {results.scale_factor}")
print(f"Test Type: {results.test_execution_type}")
print(f"Timestamp: {results.start_time}")
print("\nExecution Summary:")
print(f"  Total queries: {len(results.query_results)}")
print(f"  Successful: {sum(1 for qr in results.query_results if qr.success)}")
print(f"  Failed: {sum(1 for qr in results.query_results if not qr.success)}")
print(f"  Geometric mean: {results.geometric_mean:.3f}s")
print(f"  Total time: {results.total_execution_time:.2f}s")

if results.data_generation_time:
    print(f"\nData Generation: {results.data_generation_time:.2f}s")
if results.data_loading_time:
    print(f"Data Loading: {results.data_loading_time:.2f}s")

print("\n📋 Query Breakdown:")
for qr in results.query_results[:5]:  # Show first 5
    status = "✅" if qr.success else "❌"
    print(f"  {status} {qr.query_name}: {qr.execution_time:.3f}s")
if len(results.query_results) > 5:
    print(f"  ... and {len(results.query_results) - 5} more queries")

## 3. Advanced Examples

### Query Subset Selection

Run specific queries for targeted testing. With SQLite's slower performance, subsets are especially useful.

In [ ]:
# Fast smoke test: Run 5 representative queries
smoke_test_queries = [1, 3, 6, 10, 14]  # Mix of simple and complex

subset_cfg = BenchmarkConfig(
    name="tpch",
    display_name="TPC-H Smoke Test",
    scale_factor=0.01,
    test_execution_type="power",
    query_numbers=smoke_test_queries,
)

print(f"🚀 Running smoke test with queries: {smoke_test_queries}\n")
subset_results = run_benchmark_lifecycle(
    benchmark_config=subset_cfg,
    database_config=db_cfg,
    system_profile=None,
    platform_config=platform_cfg,
    phases=LifecyclePhases(generate=False, load=False, execute=True),  # Reuse data
)

print(f"\n✅ Smoke test completed: {subset_results.geometric_mean:.3f}s geometric mean")
print(f"   Queries: {len(subset_results.query_results)}")
print(f"   Time saved vs full suite: ~{(1 - len(smoke_test_queries) / 22) * 100:.0f}%")

### Index Optimization

SQLite benefits greatly from proper indexing for analytical queries.

In [ ]:
print("📚 SQLite Index Strategy for TPC-H\n")
print("Recommended indexes for better analytical performance:\n")

print("1. Foreign Key Indexes (for joins):")
print("   CREATE INDEX idx_orders_custkey ON orders(o_custkey);")
print("   CREATE INDEX idx_lineitem_orderkey ON lineitem(l_orderkey);")
print("   CREATE INDEX idx_lineitem_partkey ON lineitem(l_partkey);")
print("   CREATE INDEX idx_lineitem_suppkey ON lineitem(l_suppkey);")

print("\n2. Date Range Indexes (for filters):")
print("   CREATE INDEX idx_orders_orderdate ON orders(o_orderdate);")
print("   CREATE INDEX idx_lineitem_shipdate ON lineitem(l_shipdate);")

print("\n3. Composite Indexes (for complex queries):")
print("   CREATE INDEX idx_lineitem_dates ON lineitem(l_shipdate, l_receiptdate);")
print("   CREATE INDEX idx_orders_date_status ON orders(o_orderdate, o_orderstatus);")

print("\n💡 Index Trade-offs:")
print("   ✅ Significantly speed up SELECT queries")
print("   ✅ Essential for WHERE and JOIN clauses")
print("   ❌ Slow down INSERT operations")
print("   ❌ Increase database file size")
print("   ❌ Not useful for full table scans")

print("\n🎯 Best Practices:")
print("   - Index columns used in WHERE clauses")
print("   - Index foreign keys for join performance")
print("   - Use EXPLAIN QUERY PLAN to verify index usage")
print("   - Run ANALYZE after creating indexes")
print("   - Avoid over-indexing (balance read vs write performance)")

### SQLite vs DuckDB Comparison

Compare SQLite and DuckDB performance on the same dataset.

In [ ]:
print("⚖️  SQLite vs DuckDB Comparison\n")
print("Use Case Recommendations:\n")

print("**Choose SQLite when:**")
print("✅ Embedded application database")
print("✅ Mobile or IoT devices")
print("✅ Small datasets (<1GB)")
print("✅ Transactional workloads (OLTP)")
print("✅ Simple queries")
print("✅ Maximum portability (single file)")
print("✅ Mature, battle-tested stability")

print("\n**Choose DuckDB when:**")
print("✅ Analytical workloads (OLAP)")
print("✅ Large datasets (>1GB)")
print("✅ Complex aggregations and joins")
print("✅ Data science / pandas integration")
print("✅ Multi-threaded performance needed")
print("✅ Parquet/CSV querying")
print("✅ 10-100x faster for analytics")

print("\n📊 Performance Expectations (TPC-H SF 0.01):")
print(f"SQLite:  ~{results.geometric_mean:.2f}s geometric mean")
print(f"DuckDB:  ~{results.geometric_mean / 10:.2f}s expected (10x faster)")

print("\n💡 Hybrid Approach:")
print("   Use SQLite for application data (OLTP)")
print("   Export to Parquet for analysis with DuckDB (OLAP)")
print("   Best of both worlds!")

### Export Results

Export benchmark results to various formats.

In [ ]:
# Export to multiple formats
try:
    exporter = ResultExporter(results)

    output_dir = Path(config["output_dir"])
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Export to JSON
    json_path = output_dir / f"sqlite_tpch_{timestamp}.json"
    exporter.to_json(json_path)
    print(f"✅ Exported JSON: {json_path}")

    # Export to CSV
    csv_path = output_dir / f"sqlite_tpch_{timestamp}.csv"
    exporter.to_csv(csv_path)
    print(f"✅ Exported CSV: {csv_path}")

    # Export to HTML report
    html_path = output_dir / f"sqlite_tpch_{timestamp}.html"
    exporter.to_html(html_path)
    print(f"✅ Exported HTML: {html_path}")

    print(f"\n📁 All results exported to: {output_dir}")

except Exception as e:
    print(f"⚠️ Export failed: {e}")

## 4. Platform-Specific Features

### PRAGMA Commands

SQLite's PRAGMA commands control database behavior and performance.

In [ ]:
conn = sqlite3.connect(config["database_file"])
cursor = conn.cursor()

print("⚙️  Important PRAGMA Commands\n")

# Get current settings
pragmas = {
    "journal_mode": cursor.execute("PRAGMA journal_mode;").fetchone()[0],
    "synchronous": cursor.execute("PRAGMA synchronous;").fetchone()[0],
    "cache_size": cursor.execute("PRAGMA cache_size;").fetchone()[0],
    "temp_store": cursor.execute("PRAGMA temp_store;").fetchone()[0],
    "mmap_size": cursor.execute("PRAGMA mmap_size;").fetchone()[0],
}

print("Current Settings:")
for key, value in pragmas.items():
    print(f"   {key}: {value}")

print("\n💡 Key PRAGMAs for Analytics:\n")

print("1. journal_mode = WAL")
print("   Write-Ahead Logging for better concurrency")
print("   Allows concurrent readers during writes")

print("\n2. synchronous = NORMAL (or OFF for benchmarks)")
print("   Balance between safety and speed")
print("   OFF is fastest but risks corruption on crash")

print("\n3. cache_size = -64000 (64MB in KB)")
print("   Larger cache = better performance")
print("   Negative value = size in KB, positive = pages")

print("\n4. temp_store = MEMORY")
print("   Keep temporary tables in RAM")
print("   Faster for complex queries with temp results")

print("\n5. mmap_size = 30000000000 (30GB)")
print("   Memory-mapped I/O for faster reads")
print("   Lets OS cache database pages")

conn.close()

print("\n🎯 Optimization Strategy:")
print("   - Set PRAGMAs at connection time")
print("   - Run ANALYZE after loading data")
print("   - Use EXPLAIN QUERY PLAN to verify execution")
print("   - Monitor with PRAGMA stats")

### EXPLAIN QUERY PLAN

Understand how SQLite executes queries.

In [ ]:
conn = sqlite3.connect(config["database_file"])
cursor = conn.cursor()

print("🔍 Query Execution Plans\n")

# Example query
query = """
SELECT c_name, SUM(o_totalprice) as total
FROM customer
JOIN orders ON c_custkey = o_custkey
WHERE o_orderdate >= '1995-01-01'
GROUP BY c_custkey, c_name
ORDER BY total DESC
LIMIT 10;
"""

print("Example Query:")
print(query)

try:
    # Get query plan
    plan = cursor.execute(f"EXPLAIN QUERY PLAN {query}").fetchall()

    print("\nQuery Plan:")
    for row in plan:
        # Format: (id, parent, notused, detail)
        indent = "  " * row[0]
        print(f"{indent}{row[3]}")

except Exception as e:
    print(f"\n⚠️ Could not get query plan: {e}")
    print("   (This is expected if tables don't exist yet)")

conn.close()

print("\n💡 Understanding the Plan:\n")
print("- SCAN TABLE: Full table scan (slow, needs index)")
print("- SEARCH TABLE USING INDEX: Using an index (fast)")
print("- USE TEMP B-TREE FOR: Temporary sorting/grouping")
print("- COVERING INDEX: Index contains all needed columns (fastest)")

print("\n🎯 Optimization Tips:")
print("   1. Avoid SCAN operations on large tables")
print("   2. Create indexes for frequently filtered columns")
print("   3. Consider covering indexes for common queries")
print("   4. Run ANALYZE to update query planner statistics")

### ATTACH DATABASE

SQLite can attach multiple database files for cross-database queries.

In [ ]:
print("🔗 ATTACH DATABASE Feature\n")
print("SQLite can work with multiple database files simultaneously:\n")

print("1. Attach another database:")
print("   ATTACH DATABASE 'archive.db' AS archive;")

print("\n2. Query across databases:")
print("   SELECT * FROM main.customers")
print("   JOIN archive.orders ON customers.id = orders.customer_id;")

print("\n3. Copy data between databases:")
print("   INSERT INTO archive.old_orders SELECT * FROM main.orders")
print("   WHERE order_date < '2020-01-01';")

print("\n4. Detach when done:")
print("   DETACH DATABASE archive;")

print("\n💡 Use Cases:")
print("   - Separate hot/cold data (current vs archive)")
print("   - Data migration between databases")
print("   - Multi-tenant applications")
print("   - Backup and restore operations")

print("\n⚠️  Limitations:")
print("   - All databases must use same SQLite version")
print("   - Transactions can span attached databases")
print("   - Performance: Keep total data size reasonable")

## 5. Performance Analysis

### Statistical Analysis

Calculate detailed statistics on query performance.

In [ ]:
if results.query_results:
    times = [qr.execution_time for qr in results.query_results if qr.success]

    if times:
        stats = {
            "count": len(times),
            "mean": np.mean(times),
            "median": np.median(times),
            "std": np.std(times),
            "min": np.min(times),
            "max": np.max(times),
            "p95": np.percentile(times, 95),
        }

        print("📊 Statistical Summary:\n")
        print(f"Count:      {stats['count']} queries")
        print(f"Mean:       {stats['mean']:.3f}s")
        print(f"Median:     {stats['median']:.3f}s")
        print(f"Std Dev:    {stats['std']:.3f}s")
        print(f"Min:        {stats['min']:.3f}s")
        print(f"Max:        {stats['max']:.3f}s")
        print(f"P95:        {stats['p95']:.3f}s")

        print("\n💡 SQLite Performance Context:")
        print("   SQLite is single-threaded and OLTP-optimized")
        print("   These times are expected for analytical workloads")
        print("   For faster analytics, consider DuckDB or cloud warehouses")
    else:
        print("⚠️  No successful queries to analyze")
else:
    print("⚠️  No query results available")

## 6. Troubleshooting

### Diagnostics Function

Comprehensive diagnostic tool for troubleshooting SQLite issues.

In [ ]:
def diagnose_sqlite():
    """Diagnose SQLite setup and configuration"""
    print("🔍 SQLite Diagnostic\n")

    # Check 1: SQLite version
    print("1. Checking SQLite version...")
    print(f"   ✅ SQLite {sqlite3.sqlite_version}")
    print(f"   ✅ Python sqlite3 {sqlite3.version}")

    # Check 2: Test connection
    print("\n2. Testing connection...")
    try:
        conn = sqlite3.connect(":memory:")
        result = conn.execute("SELECT 42 as answer;").fetchone()
        conn.close()
        print(f"   ✅ Connection successful: {result}")
    except Exception as e:
        print(f"   ❌ Connection failed: {e}")
        return False

    # Check 3: Database file
    print("\n3. Checking database file...")
    db_path = Path(config["database_file"])
    if db_path.exists():
        size_mb = db_path.stat().st_size / (1024**2)
        print(f"   ✅ Database exists: {size_mb:.1f} MB")

        # Check if file is locked
        try:
            test_conn = sqlite3.connect(config["database_file"])
            test_conn.close()
            print("   ✅ Database accessible")
        except sqlite3.OperationalError as e:
            print(f"   ❌ Database locked: {e}")
            return False
    else:
        print(f"   ℹ️  Database will be created: {db_path}")

    # Check 4: Compile options
    print("\n4. Checking compile options...")
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    options = cursor.execute("PRAGMA compile_options;").fetchall()

    key_features = ["THREADSAFE", "ENABLE_FTS", "ENABLE_JSON", "ENABLE_RTREE"]
    for (option,) in options:
        for feature in key_features:
            if feature in option:
                print(f"   ✅ {option}")

    conn.close()

    print("\n✅ All diagnostics passed!")
    print("\n📚 Resources:")
    print("   - SQLite Documentation: https://sqlite.org/docs.html")
    print("   - SQL As Understood By SQLite: https://sqlite.org/lang.html")
    print("   - Performance Tuning: https://sqlite.org/speed.html")

    return True


# Run diagnostics
diagnose_sqlite()

### Common Issues and Solutions

**1. Database Locked Error**
```
sqlite3.OperationalError: database is locked
```
**Solution:**
- Close all connections: `conn.close()`
- Use WAL mode: `PRAGMA journal_mode=WAL;`
- Increase timeout: `conn = sqlite3.connect(db, timeout=30)`
- Check for stale lock files (delete .db-journal if safe)

**2. Slow Query Performance**
**Solution:**
- Create indexes on filtered/joined columns
- Run ANALYZE to update statistics
- Increase cache_size: `PRAGMA cache_size=-64000;`
- Use EXPLAIN QUERY PLAN to find bottlenecks
- Consider DuckDB for complex analytical queries

**3. Disk I/O Error**
**Solution:**
- Check disk space available
- Verify file permissions
- Run integrity check: `PRAGMA integrity_check;`
- Backup and restore if corruption detected

**4. Out of Memory**
**Solution:**
- Reduce cache_size setting
- Process data in smaller batches
- Use temp_store=FILE instead of MEMORY
- Simplify complex queries

**5. Syntax Error (Unsupported SQL)**
**Solution:**
- SQLite has limited SQL support vs PostgreSQL/MySQL
- No RIGHT JOIN (use LEFT JOIN instead)
- No FULL OUTER JOIN (union two LEFT JOINs)
- Limited window functions in older versions
- Check version: `SELECT sqlite_version();`

**6. Slow Bulk Insert**
**Solution:**
```python
conn.execute('BEGIN TRANSACTION')
# Insert many rows
conn.execute('COMMIT')
```
- Wrap inserts in transaction
- Use executemany() for batch inserts
- Temporarily disable indexes
- Set `PRAGMA synchronous=OFF` (careful!)

**7. Database Corruption**
**Solution:**
- Run: `PRAGMA integrity_check;`
- Dump and restore: `.dump | sqlite3 new.db`
- Use WAL mode to prevent corruption
- Always close connections properly

**8. Large Database File Size**
**Solution:**
- Run: `VACUUM;` to reclaim space
- Enable auto_vacuum: `PRAGMA auto_vacuum=FULL;`
- Delete unnecessary indexes
- Archive old data to separate database

**Need More Help?**
- SQLite Documentation: https://sqlite.org/docs.html
- SQLite Forum: https://sqlite.org/forum/
- Stack Overflow: https://stackoverflow.com/questions/tagged/sqlite

## Next Steps

**Try these next:**
1. Compare with DuckDB performance (`duckdb_benchmarking.ipynb`)
2. Test with your own application data
3. Experiment with index strategies
4. Optimize PRAGMA settings for your workload
5. Consider SQLite for OLTP, DuckDB for OLAP

**Resources:**
- BenchBox Documentation: https://github.com/joeharris76/benchbox
- SQLite Documentation: https://sqlite.org/docs.html
- SQLite Performance Tips: https://sqlite.org/speed.html
- TPC Benchmarks: http://www.tpc.org/